#딥러닝 학슬 프레임워크를 사용하는 이유는 결국 신경망을 사용하기 위함이다. 드디어 파이토치에서 신경망을 다루는 튜토리얼이다. 텐서플로우와 크게 다를거라 생각치는 않지만, 문법적인 부분은 다를수도 있으니 참고해야겠다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습 장치를 선택하기...
CUDA 혹은 CPU가 있을 수 있다. 

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


## 객체? 클래스 정의하기
텐서플로우는 따로 신경망을 객체로 사용하지 않았던 것같은데, 파이토치는 신경망을 객체로 사용한다. nn.Module 객체를 상속하기 때문에 대부분의 메소드가 원래 구현되어 있을 것으로 예상되나, 필요한 경우 오버라이드하여 사용이 가능할 것 같다.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        # 왜 부모 객체를 먼저 생성하는지 의문... Why?
        super(NeuralNetwork, self).__init__() # 여튼 부모 클래스를 먼저 생성하는데 여기서도 self가 뒤에 나옴... 흠..?
        # 객체 지역 변수를 이렇게 설정했는데... 어떻게 외부에서 이 변수에 접근할 수 있는거지?
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

위 코드는 짧은 코드라고 볼 수 있겠지만, 많은 내용이 숨겨져있는 것 같음.. 깊에 파고 든다면 어렵겠지만 여튼 레이어를 위와 같이 구성한다는 것은 알 수 있었음.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


현재 우리가 정의한 NueralNetwork라는 이름의 객체 변수가 생성된 상태이다. 이 변수에 어떻게 변수를 입력하는지 아래 코드로 확인할 수 있다.

In [11]:
X = torch.rand(1, 28, 28, device=device) #파이토치의 랜덤은 곧 바로 텐서 형태로 반환하기 때문에 사용하는 것으로 보임.
logits = model(X) # 우리가 사용하는 변수에 X 입력을 넣기 위해서는 이렇게 사용하는 것으로 보임.
print(f"logits : {logits}")

pred_probab = nn.Softmax(dim=1)(logits)
print(f"Softmaxed prob : {pred_probab}")

y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

logits : tensor([[0.0772, 0.0000, 0.0664, 0.0000, 0.0000, 0.0000, 0.0000, 0.0705, 0.0000,
         0.0000]], grad_fn=<ReluBackward0>)
Softmaxed prob : tensor([[0.1057, 0.0978, 0.1045, 0.0978, 0.0978, 0.0978, 0.0978, 0.1050, 0.0978,
         0.0978]], grad_fn=<SoftmaxBackward>)
Predicted class: tensor([0])


아래 블럭에서는 mini batch 사이즈를 어떻게 조절하는지 나타낸다. 한번 알아보자

In [13]:
input_image = torch.rand(3,28,28) # 패션 MNIST 데이터 셋에서는 입력 이미지가 그레이스케일인 것 같다. 따라서 디멘젼이 1이고 28 * 28 사이즈의 이미지가 입력되며, 여기서는 배치가 3이라서 3장의 사진이 입력된다.
print(input_image.size())

torch.Size([3, 28, 28])


2차원 행렬 형태로 학습을 진행할 수 있지만, 그건 CNN이고 예제는 MLP로 진행하니 우선 입력 데이터를 평평하게 만들어보자.

In [14]:
flatten = nn.Flatten() # 우리가 정의한 모델의 사용법과 마찬가지로, 플랫 네트워크를 선언하고 해당 변수에 입력을 넣는 형태로 데이터를 가공할 수 있다.
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


입력 피처, 출력 피처의 갯 수에 맞게끔 선형 출력(변환? 결합? 조합?)하는 코드는 다음과 같다.

In [17]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


선형 변환이후 ReLU 레이어를 통해서 값에 비선형성을 추가하고 정리하는 과정의 코드는 다음과 같다.

In [18]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.6855,  0.1541,  0.2985, -0.1219, -0.0351, -0.0583,  1.0957,  0.3237,
         -0.0323,  0.4022,  0.0072,  0.2434,  0.4087, -0.2348,  0.1923, -0.0541,
          0.0744,  0.5194, -0.1346,  0.0074],
        [-0.7354,  0.2630,  0.8224, -0.0334,  0.1123,  0.0686,  0.6602,  0.0477,
         -0.0419,  0.0280,  0.0978,  0.2687,  0.3399, -0.1708,  0.1353,  0.0620,
         -0.1466, -0.1236, -0.0727,  0.0437],
        [-0.7506,  0.1862,  0.4261,  0.2827, -0.2261,  0.4101,  0.5261,  0.3174,
         -0.1297,  0.0909, -0.1734,  0.2590,  0.3615, -0.1948,  0.0826, -0.0764,
         -0.4700, -0.0195,  0.3486, -0.0857]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.1541, 0.2985, 0.0000, 0.0000, 0.0000, 1.0957, 0.3237, 0.0000,
         0.4022, 0.0072, 0.2434, 0.4087, 0.0000, 0.1923, 0.0000, 0.0744, 0.5194,
         0.0000, 0.0074],
        [0.0000, 0.2630, 0.8224, 0.0000, 0.1123, 0.0686, 0.6602, 0.0477, 0.0000,
         0.0280, 0.0978, 0.2687, 0.3399, 0.0000, 0.135

위와 같이 파이토치 nn 패키지 하위의 여러 객체를 이용하여 레이어를 구성할 수 있다. 하지만 위와 같은 방법을 조금 더 간편하게 사용하게끔 동작하는 순서대로 레이어를 쌓도록하는 시퀀셜 객체를 지원한다. 코드는 다음과 같다.

In [20]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

원래는 Softmax도 시퀀셜 레이어에 추가할 수 있을 텐데 여기서는 따른 레이어로 구성했다. 코드는 아래와 같다.

In [21]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

레이어에 대한 정보를 출력할 수 있다. 파이토치의 경우 대부분의 변수들이 자동으로 관리된다고 한다. 변수 이름과 왜 쓰이는지를 확실하게 알아두고 나중에 사용할 때 유의하면 되겠다...

In [22]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0115,  0.0257,  0.0073,  ..., -0.0310,  0.0059, -0.0022],
        [ 0.0034,  0.0208,  0.0284,  ..., -0.0257, -0.0038,  0.0239]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0325, -0.0097], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0366,  0.0066,  0.0123,  ..., -0.0315, -0.0416,  0.0405],
        [ 0.0268, -0.0358, -0.0327,  ...,  0.0126,  0.0171, -0.0206]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_